## This notebook is for testing the SDK before using any scripts for data retrieval / live feed
- Make sure you have the .env file with the corrent variable names listed below before running the code

In [2]:
# Authentication for streaming data
from neurosity import NeurositySDK
from dotenv import load_dotenv
import os

load_dotenv()

neurosity = NeurositySDK({
    "device_id": os.getenv("NEUROSITY_DEVICE_ID")
})

neurosity.login({
    "email": os.getenv("NEUROSITY_EMAIL"),
    "password": os.getenv("NEUROSITY_PASSWORD")
})

In [ ]:
# checking the info from the SDK
info = neurosity.get_info()
print(info)

OrderedDict({'channelNames': ['CP6', 'F6', 'C4', 'CP4', 'CP3', 'F5', 'C3', 'CP5'], 'channels': 8, 'deviceId': 'e425ea92bcd7bdae65a5c2b07322433d', 'deviceNickname': 'Emulator-E42', 'emulator': True, 'manufacturer': 'Neurosity, Inc', 'model': 'Emulator 2', 'modelName': 'Emulator', 'modelVersion': '2', 'osVersion': '14.5.0', 'samplingRate': 250})


In [6]:
# Retrieving raw brainwaves for 5 seconds
import time


def callback(data):
    print("data", data)
    
unsubscribe = neurosity.brainwaves_raw(callback)

time.sleep(5)

unsubscribe()

data {'data': [[0.8823863863928061, 17.725217682247358, 29.50683475243079, 30.507778998651368, 20.941853859682425, 8.16559828368825, 0.5107786511377497, 1.9161822409506621, 9.607362133351664, 15.072277557598081, 12.751636656850804, 4.530352373222021, -3.5778463146397055, -4.723447051732467, 2.4509910597047346, 10.128153004984783, 10.004872040151415, 1.723591776826403, -8.239906371227544, -11.946488596268418, -7.247378478133319, -1.089365308670069, -2.7873650588489225, -12.747657202710936, -20.990733236340162], [-1.6754587677081283, -2.543023517477502, -3.3621690076536623, -3.9376832885650925, -5.705752492780903, -9.131626227335655, -13.68848319956746, -16.73764923665125, -15.450033585039373, -10.805081282496063, -7.605293500130175, -8.874696803020926, -11.28893307152522, -9.992261067267004, -3.980176795477967, 4.004767411070832, 9.102518020755934, 10.385165509939952, 11.679493180743926, 15.197057472249698, 20.725124987573054, 25.551412164011698, 26.123881988745325, 21.55523421334913, 1